In [21]:
!python -m spacy download en_core_web_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     --------------------------------------- 0.1/12.8 MB 825.8 kB/s eta 0:00:16
      --------------------------------------- 0.3/12.8 MB 2.3 MB/s eta 0:00:06
     - -------------------------------------- 0.5/12.8 MB 3.4 MB/s eta 0:00:04
     - -------------------------------------- 0.6/12.8 MB 3.1 MB/s eta 0:00:04
     -- ------------------------------------- 0.9/12.8 MB 3.8 MB/s eta 0:00:04
     ----- ---------------------------------- 1.7/12.8 MB 5.7 MB/s eta 0:00:02
     ------ --------------------------------- 2.2/12.8 MB 6.3 MB/s eta 0:00:02
     -------- ------------------------------- 2.6/12.8 MB 6.9 MB/s eta 0:00:02
     --------- ------------------------------ 3.1/12.8 MB 7.1 MB/s eta 0:00:02
     ----------- ---------------------------- 3.6/12.8 MB 7.4 MB/s eta 0:00:02
     ------------ --------------------------- 4.0/12.8 MB 7.6 MB/

DEPRECATION: Loading egg at c:\users\micha\anaconda3\lib\site-packages\huggingface_hub-0.23.0-py3.8.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..


In [22]:
import gensim
import gensim, spacy, logging, warnings
import gensim.corpora as corpora
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models import CoherenceModel
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np
import pickle

In [4]:
#use wordnet lemmatizor, gensims lemmatizer is deactivated
from nltk.stem import WordNetLemmatizer

In [5]:
#read in file
with open('../data/data_simple_preprocess/complete_corpus_simple_preprocess_sw_removed.txt', 'rb') as file:
    complete_corpus_simple_preprocess_sw_removed = pickle.load(file)

In [6]:
wnl = WordNetLemmatizer()

In [7]:

complete_corpus_simple_preprocess={}

for key, value in complete_corpus_simple_preprocess_sw_removed.items():
    a_string = ''.join(value)
    lemmatized=' '.join([wnl.lemmatize(word) for word in a_string.split()]) #cant lemmatize over a simple call.  Need a list comprehension, similar to the tagged docs
    complete_corpus_simple_preprocess[key]=lemmatized

In [8]:
complete_corpus_simple_preprocess_sw_removed

{'Address of Tatian to the Greeks': ['address',
  'tatian',
  'greeks',
  'chapter',
  'the',
  'greeks',
  'claim',
  'without',
  'reason',
  'invention',
  'arts',
  'be',
  'not',
  'greeks',
  'hostilely',
  'disposed',
  'barbarians',
  'look',
  'ill',
  'opinions',
  'for',
  'institutions',
  'derived',
  'barbarians',
  'the',
  'eminent',
  'telmessians',
  'invented',
  'art',
  'divining',
  'dreams',
  'carians',
  'prognosticating',
  'stars',
  'phrygians',
  'ancient',
  'isaurians',
  'augury',
  'flight',
  'birds',
  'cyprians',
  'art',
  'inspecting',
  'victims',
  'to',
  'babylonians',
  'owe',
  'astronomy',
  'persians',
  'magic',
  'egyptians',
  'geometry',
  'phoenicians',
  'instruction',
  'alphabetic',
  'writing',
  'cease',
  'then',
  'miscall',
  'imitations',
  'inventions',
  'own',
  'orpheus',
  'again',
  'taught',
  'poetry',
  'song',
  'him',
  'too',
  'learned',
  'mysteries',
  'the',
  'tuscans',
  'taught',
  'plastic',
  'art',
  'ann

In [12]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(complete_corpus_simple_preprocess_sw_removed.values(), min_count=5, threshold=50) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[complete_corpus_simple_preprocess_sw_removed.values()], threshold=50)  
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [ ]:

# !python3 -m spacy download en  # run in terminal once, then hash out?
def process_words(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """Remove Stopwords, Form Bigrams, Trigrams and Lemmatization"""
    texts = [bigram_mod[doc] for doc in texts]
    texts = [trigram_mod[bigram_mod[doc]] for doc in texts]
    texts_out = []
    nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    # remove stopwords once more after lemmatization
    texts_out = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts_out]    
    return texts_out

data_ready = process_words(complete_corpus_simple_preprocess_sw_removed.values())  # processed Text Data!

In [ ]:
# Create Dictionary
id2word = corpora.Dictionary(data_ready)

# Create Corpus: Term Document Frequency
corpus = [id2word.doc2bow(text) for text in data_ready]

# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=4, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=10,
                                           passes=10,
                                           alpha='symmetric',
                                           iterations=100,
                                           per_word_topics=True)

pprint(lda_model.print_topics())